# 🧪 Codelab: Bernstein-Vazirani Algorithm

| Metadata | Value |
|----------|-------|
| **Algorithm** | Bernstein-Vazirani |
| **Difficulty** | 🟡 Intermediate |
| **Time** | 60-90 minutes |
| **Prerequisites** | Deutsch-Jozsa Algorithm, Hadamard gates, CNOT |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Construct oracles for f(x) = s·x mod 2
2. ✅ Implement the Bernstein-Vazirani algorithm
3. ✅ Understand bit ordering in Qiskit
4. ✅ Compare with classical query approach
5. ✅ Analyze algorithm behavior under noise

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.primitives import StatevectorSampler as SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### The Problem

An oracle implements the function $f(x) = s \cdot x \pmod{2}$ where:
- $s$ is an unknown n-bit **secret string**
- $x$ is an n-bit input
- $s \cdot x = s_0 x_0 \oplus s_1 x_1 \oplus \cdots \oplus s_{n-1} x_{n-1}$ (bitwise dot product)

**Goal**: Find the secret string $s$.

### Classical vs Quantum

| Approach | Queries Needed |
|----------|---------------|
| Classical | n queries (one per bit) |
| **Quantum (BV)** | **1 query** |

### The Magic: Hadamard Decoding

After the oracle, the state has phases encoding $s$:
$$|\psi\rangle = \frac{1}{\sqrt{2^n}}\sum_{x}(-1)^{s \cdot x}|x\rangle$$

The Hadamard transform **decodes** this to:
$$H^{\otimes n}|\psi\rangle = |s\rangle$$

The secret appears directly in the measurement!

## Section 3: Basic Implementation - The BV Oracle

In [ ]:
def create_bv_oracle(secret: str) -> QuantumCircuit:
    """
    Create a Bernstein-Vazirani oracle for the given secret string.
    
    The oracle implements f(x) = s·x mod 2 where s is the secret.
    
    Args:
        secret: Binary string representing the secret (e.g., "1011")
    
    Returns:
        QuantumCircuit implementing the oracle
    
    Implementation:
        For each bit position i where s_i = 1, add CNOT from qubit i to ancilla.
        This computes s·x by XORing the bits where s has 1s.
    """
    n = len(secret)
    oracle = QuantumCircuit(n + 1, name=f"Oracle(s={secret})")
    
    # IMPORTANT: Qiskit uses LSB ordering (qubit 0 = rightmost bit)
    # secret[0] is MSB, so we need to reverse the indexing
    for i, bit in enumerate(reversed(secret)):
        if bit == '1':
            oracle.cx(i, n)  # CNOT from input qubit i to ancilla
    
    return oracle


# Demonstrate oracle construction
secrets_to_demo = ["110", "101", "1111", "1001"]

for secret in secrets_to_demo:
    oracle = create_bv_oracle(secret)
    print(f"Oracle for secret s = {secret}:")
    print(oracle.draw())
    print()

### Understanding the Oracle

For secret $s = 110$:
- $s_2 = 1$ (MSB) → CNOT from $q_2$ to ancilla
- $s_1 = 1$ → CNOT from $q_1$ to ancilla  
- $s_0 = 0$ (LSB) → No CNOT from $q_0$

The XOR of controlled bits computes $s \cdot x$.

In [ ]:
def verify_oracle_truth_table(oracle: QuantumCircuit, secret: str) -> bool:
    """
    Verify that the oracle correctly implements f(x) = s·x mod 2.
    
    Args:
        oracle: The oracle circuit
        secret: The secret string
    
    Returns:
        True if all outputs match expected values
    """
    n = len(secret)
    sampler = SamplerV2()
    backend = FakeManilaV2()
    
    print(f"Verifying oracle for s = {secret}")
    print("="*40)
    print(f"{'Input x':<12} {'s·x':<8} {'Oracle output':<15} {'Match'}")
    print("-"*40)
    
    all_correct = True
    
    for x_val in range(2**n):
        x_str = format(x_val, f'0{n}b')
        
        # Calculate expected s·x mod 2
        s_dot_x = 0
        for i, (s_bit, x_bit) in enumerate(zip(secret, x_str)):
            s_dot_x ^= (int(s_bit) & int(x_bit))
        
        # Run oracle
        qc = QuantumCircuit(n + 1, 1)
        # Prepare |x⟩|0⟩
        for i, bit in enumerate(reversed(x_str)):
            if bit == '1':
                qc.x(i)
        # Apply oracle
        qc.compose(oracle, inplace=True)
        # Measure ancilla
        qc.measure(n, 0)
        # Transpile and run with SamplerV2
        transpiled_qc = transpile(qc, backend=backend, optimization_level=1)
        job = sampler.run([transpiled_qc], shots=1)
        result = job.result()
        output = int(list(result[0].data.c.get_counts().keys())[0])
        
        match = "✓" if output == s_dot_x else "✗"
        if output != s_dot_x:
            all_correct = False
        
        print(f"{x_str:<12} {s_dot_x:<8} {output:<15} {match}")
    
    print("="*40)
    return all_correct


# Verify an oracle
secret = "110"

oracle = create_bv_oracle(secret)

correct = verify_oracle_truth_table(oracle, secret)
print(f"\nOracle verification: {'PASSED' if correct else 'FAILED'}")

## Section 4: Complete Bernstein-Vazirani Algorithm

In [ ]:
def bernstein_vazirani(secret: str) -> QuantumCircuit:
    """
    Construct the complete Bernstein-Vazirani algorithm circuit.
    
    Args:
        secret: Binary string representing the secret
    
    Returns:
        Complete BV circuit with measurements
    
    Algorithm:
        1. Initialize |0⟩^n|1⟩
        2. Apply H^(n+1)
        3. Apply oracle U_s
        4. Apply H^n to input qubits
        5. Measure input qubits → result is s
    """
    n = len(secret)
    oracle = create_bv_oracle(secret)
    
    # Create registers with meaningful names
    qr_input = QuantumRegister(n, 'input')
    qr_ancilla = QuantumRegister(1, 'ancilla')
    cr = ClassicalRegister(n, 'secret')
    qc = QuantumCircuit(qr_input, qr_ancilla, cr)
    
    # Step 1: Prepare ancilla in |1⟩
    qc.x(qr_ancilla[0])
    
    # Step 2: Apply Hadamard to all qubits
    qc.h(qr_input)
    qc.h(qr_ancilla[0])
    
    qc.barrier(label="init")
    
    # Step 3: Apply oracle
    qc.compose(oracle, inplace=True)
    
    qc.barrier(label="oracle")
    
    # Step 4: Apply Hadamard to input qubits
    qc.h(qr_input)
    
    qc.barrier(label="decode")
    
    # Step 5: Measure input qubits
    qc.measure(qr_input, cr)
    
    return qc


# Build and display the circuit
secret = "1011"
bv_circuit = bernstein_vazirani(secret)
print(f"Bernstein-Vazirani circuit for secret s = {secret}:")
print(bv_circuit.draw(fold=100))

In [ ]:
# Run the algorithm and verify it finds the secret
def find_secret(secret: str, shots: int = 1) -> str:
    """
    Run BV algorithm to find the secret string.
    
    Args:
        secret: The actual secret (for oracle construction)
        shots: Number of measurement shots
    
    Returns:
        The discovered secret string
    """
    circuit = bernstein_vazirani(secret)
    
    # Use SamplerV2 (ideal simulator, no backend constraints)
    sampler = SamplerV2()
    job = sampler.run([circuit], shots=shots)
    result = job.result()
    
    # Access the measurement data correctly for SamplerV2
    counts = result[0].data.secret.get_counts()
    
    # Get the most common measurement
    measured = max(counts, key=counts.get)
    return measured, counts


# Test with several secrets
test_secrets = ["0", "1", "11", "101", "1100", "11111", "10101010"]

print("Bernstein-Vazirani Algorithm Results")
print("=" * 50)
print(f"{'Secret':<15} {'Found':<15} {'Match'}")
print("-" * 50)

for secret in test_secrets:
    found, counts = find_secret(secret)
    match = "✓" if found == secret else "✗"
    print(f"{secret:<15} {found:<15} {match}")

print("\n💡 Note: BV finds the secret in exactly ONE query!")
print("=" * 50)


## Section 5: State Evolution Visualization

In [ ]:
def visualize_bv_evolution(secret: str):
    """
    Visualize the quantum state at each step of the BV algorithm.
    """
    n = len(secret)
    oracle = create_bv_oracle(secret)
    
    print(f"\n{'='*60}")
    print(f"STATE EVOLUTION for secret s = {secret}")
    print("="*60)
    
    # Build circuit step by step
    qc = QuantumCircuit(n + 1)
    
    # Step 0: Initial state
    state = Statevector(qc)
    print(f"\nStep 0 - Initial |{'0'*n}⟩|0⟩:")
    print(f"  State: {state}")
    
    # Step 1: X on ancilla
    qc.x(n)
    state = Statevector(qc)
    print(f"\nStep 1 - After X on ancilla |{'0'*n}⟩|1⟩:")
    print(f"  State: {state}")
    
    # Step 2: Hadamard on all
    qc.h(range(n + 1))
    state = Statevector(qc)
    print(f"\nStep 2 - After H^(n+1) (superposition):")
    print(f"  Uniform superposition of {2**n} input states ⊗ |−⟩")
    print(f"  Number of non-zero amplitudes: {sum(1 for a in state.data if abs(a) > 1e-10)}")
    
    # Step 3: Oracle
    qc.compose(oracle, inplace=True)
    state = Statevector(qc)
    print(f"\nStep 3 - After Oracle (phase encoding):")
    print(f"  Phases now encode s·x for each |x⟩")
    # Show a few amplitudes
    print(f"  Sample amplitudes:")
    for i in range(min(4, 2**(n+1))):
        if abs(state.data[i]) > 1e-10:
            sign = "+" if state.data[i].real > 0 else "-"
            print(f"    |{format(i, f'0{n+1}b')}⟩: {sign}1/√{2**(n+1)}")
    
    # Step 4: Hadamard on input qubits
    qc.h(range(n))
    state = Statevector(qc)
    print(f"\nStep 4 - After final H^n (decoding):")
    probs = state.probabilities()
    
    # Find the dominant state
    for i in range(len(probs)):
        if probs[i] > 0.01:  # Significant probability
            binary = format(i, f'0{n+1}b')
            input_bits = binary[:-1]  # Remove ancilla
            print(f"    |{binary}⟩ (input: {input_bits}): P = {probs[i]:.4f}")
    
    # What should we measure?
    print(f"\n  📊 Expected measurement: {secret}")
    
    return state


# Visualize for different secrets
for secret in ["11", "101"]:
    visualize_bv_evolution(secret)

In [ ]:
# Visualize the probability distribution
def plot_bv_probabilities(secret: str):
    """Plot the final state probabilities."""
    n = len(secret)
    circuit = bernstein_vazirani(secret)
    
    # Remove measurements to get statevector
    circuit_no_meas = QuantumCircuit(n + 1)
    circuit_no_meas.x(n)
    circuit_no_meas.h(range(n + 1))
    circuit_no_meas.compose(create_bv_oracle(secret), inplace=True)
    circuit_no_meas.h(range(n))
    
    state = Statevector(circuit_no_meas)
    probs = state.probabilities()
    
    # Plot input qubit probabilities (trace out ancilla)
    input_probs = np.zeros(2**n)
    for i in range(2**(n+1)):
        input_idx = i >> 1  # Remove ancilla bit (it's the MSB in Qiskit ordering)
        input_probs[input_idx] += probs[i]
    
    fig, ax = plt.subplots(figsize=(10, 5))
    labels = [format(i, f'0{n}b') for i in range(2**n)]
    colors = ['green' if labels[i] == secret else 'lightblue' for i in range(2**n)]
    
    ax.bar(range(2**n), input_probs, color=colors, edgecolor='black')
    ax.set_xlabel('Measurement Outcome (Input Qubits)')
    ax.set_ylabel('Probability')
    ax.set_title(f'Bernstein-Vazirani Final State (secret s = {secret})')
    ax.set_xticks(range(2**n))
    ax.set_xticklabels(labels, rotation=45 if n > 4 else 0)
    ax.set_ylim(0, 1.1)
    
    # Add annotation
    secret_idx = int(secret, 2)
    ax.annotate(f'P(|{secret}⟩) = 1.0\n(100% certainty)', 
                xy=(secret_idx, 1.0), xytext=(secret_idx, 1.05),
                ha='center', fontsize=10, color='green',
                arrowprops=dict(arrowstyle='->', color='green'))
    
    plt.tight_layout()
    plt.show()


# Plot for a secret
plot_bv_probabilities("1011")

---

## Section 5.5: State Evolution Analysis (From Lecture Transcripts)

> **Source**: L1.6 Bernstein-Vazirani Algorithm Theory, L1.7 Bernstein-Vazirani Algorithm Lab

This section traces the quantum state at each stage using the key mathematical identity from the lectures.

### 5.5.1 The Hadamard Identity and the "1-Matching" Rule

The same identity from Deutsch-Jozsa applies here:

$$H^{\otimes n}|a\rangle = \frac{1}{\sqrt{2^n}} \sum_{d=0}^{2^n-1} (-1)^{a \cdot d} |d\rangle$$

where $a \cdot d = \sum_i a_i d_i \pmod{2}$ is the **bitwise inner product** (count of positions where both bits are 1, mod 2).

**The "1-Matching" Rule for Signs:**
The sign $(-1)^{a \cdot d}$ equals:
- $+1$ if the count of positions where **BOTH** $a_i = 1$ AND $d_i = 1$ is **even**
- $-1$ if the count is **odd**

**Key Insight from Lecture (L1.6):**
> "The purpose of this function [s·x mod 2] is to just check how many times you get a match of 1 and 1 at the same location between s and x. Count the number of times that happens... and give a final bit of information which is either 0 or 1 depending on whether you get even number of matches or you get odd number of matches."

**Why BV Works - The Phase Kickback Trick:**

From L1.6: "At the heart of all of this algorithm is a trick called **phase kickback**... What has happened effectively is, I was sending in |−⟩ here and |0⟩ here. The target line just went through without any change... but my control line got flipped. This is in some sense opposite of what you were aware of."

In other words:
- CNOT with target in |−⟩ state **flips the control qubit's phase**, not the target!
- This turns $|x\rangle|{-}\rangle \rightarrow (-1)^{f(x)}|x\rangle|{-}\rangle$

In [ ]:
def one_matching_sign(input_bits: str, output_bits: str) -> int:
    """
    Compute the sign using the 1-matching rule from the lecture.
    
    From L1.6: "Count how many times you get a match of 1 and 1 at the same location"
    - Even matches → +1
    - Odd matches → -1
    """
    assert len(input_bits) == len(output_bits), "Bit strings must have same length"
    matches = sum(1 for a, d in zip(input_bits, output_bits) if a == '1' and d == '1')
    return (-1) ** matches


# Verify the lecture example from L1.6
print("Lecture Example (L1.6): s·x mod 2 for s=10011")
print("=" * 55)

s = "10011"
test_cases = [
    ("00011", "Expected: 0 (even matches)"),
    ("01011", "Expected: 1 (odd matches)"),
]

for x, expected in test_cases:
    # Count 1-1 matches
    matches = sum(1 for si, xi in zip(s, x) if si == '1' and xi == '1')
    result = matches % 2
    print(f"s = {s}")
    print(f"x = {x}")
    print(f"1-1 matches: {matches} → s·x mod 2 = {result}")
    print(f"{expected}")
    print("-" * 55)

### 5.5.2 State Evolution at Each Barrier

Let's trace through the BV algorithm showing the quantum state at each stage.

**State ①** (After X on ancilla): $|\psi_1\rangle = |0\rangle^{\otimes n} \otimes |1\rangle$

**State ②** (After Hadamards): 
$$|\psi_2\rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} |x\rangle \otimes |{-}\rangle$$
- Input register: uniform superposition (all signs positive since $0 \cdot x = 0$)
- Ancilla: $H|1\rangle = |{-}\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$

**State ③** (After Oracle - Phase Kickback):
$$|\psi_3\rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} (-1)^{s \cdot x} |x\rangle \otimes |{-}\rangle$$

**This is the key step!** The oracle computes $f(x) = s \cdot x \mod 2$ via CNOTs:
- CNOT is applied at position $i$ only where $s_i = 1$
- The |−⟩ target causes **phase kickback**: flips the phase of |x⟩ when $x_i = 1$
- Total phase flip count = number of positions where both $s_i = 1$ AND $x_i = 1$
- This is exactly $s \cdot x \mod 2$!

**State ④** (After Final Hadamards - The Magic):
$$|\psi_4\rangle = \frac{1}{2^n} \sum_{z} \left[\sum_{x} (-1)^{s \cdot x + x \cdot z}\right] |z\rangle \otimes |{-}\rangle$$

Using the identity: $\sum_x (-1)^{x \cdot (s \oplus z)} = 2^n \delta_{s,z}$ (only non-zero when $z = s$)

$$|\psi_4\rangle = |s\rangle \otimes |{-}\rangle$$

**The secret string $s$ is revealed with 100% probability!**

| Function | Classical Queries | Quantum Queries | Speedup |
|----------|-------------------|-----------------|---------|
| $f(x) = s \cdot x \mod 2$ | $n$ (one per bit) | **1** | $n\times$ |

In [ ]:
def trace_bv_evolution(secret: str):
    """
    Trace state evolution through BV algorithm using 1-matching rule.
    
    From L1.6: Shows why the secret string appears directly in measurement.
    
    Parameters:
        secret: The secret bit string s
    """
    n = len(secret)
    N = 2**n
    
    print(f"="*65)
    print(f"Bernstein-Vazirani State Evolution (s = {secret}, {n} qubits)")
    print(f"="*65)
    
    # ═══════════════ State ② After Initial Hadamards ═══════════════
    print(f"\n📍 STATE ② (After Hadamards on all qubits)")
    print(f"   Input register: uniform superposition over {N} states")
    print(f"   |ψ₂⟩ = (1/√{N}) Σ |x⟩ ⊗ |−⟩")
    print(f"\n   All phases are +1 (since 0·x = 0 for the initial |0⟩ state)")
    
    # ═══════════════ State ③ After Oracle ═══════════════
    print(f"\n📍 STATE ③ (After Oracle - Phase Kickback)")
    print(f"   Oracle: f(x) = s·x mod 2 where s = {secret}")
    print(f"   |ψ₃⟩ = (1/√{N}) Σ (-1)^(s·x) |x⟩ ⊗ |−⟩")
    print(f"\n   Phase encoding via 1-matching rule:")
    
    for x in range(min(N, 8)):  # Show first 8 states
        x_bin = format(x, f'0{n}b')
        # Count 1-1 matches
        matches = sum(1 for si, xi in zip(secret, x_bin) if si == '1' and xi == '1')
        sx_mod2 = matches % 2
        sign = '+' if sx_mod2 == 0 else '-'
        print(f"      |{x_bin}⟩: s·x = {matches} mod 2 = {sx_mod2} → sign = {sign}1")
    if N > 8:
        print(f"      ... ({N-8} more states)")
    
    # ═══════════════ State ④ After Final Hadamards ═══════════════
    print(f"\n📍 STATE ④ (After Final Hadamards - Interference)")
    print(f"   Using identity: H⊗ⁿ|x⟩ = (1/√{N}) Σ_z (-1)^(x·z) |z⟩")
    print(f"\n   Amplitude of |z⟩ = (1/{N}) Σ_x (-1)^(s·x + x·z)")
    print(f"                    = (1/{N}) Σ_x (-1)^(x·(s⊕z))")
    
    # Calculate amplitudes for each possible output z
    print(f"\n   Key amplitudes:")
    for z in range(min(N, 8)):
        z_bin = format(z, f'0{n}b')
        # s XOR z
        s_xor_z = ''.join('1' if secret[i] != z_bin[i] else '0' for i in range(n))
        
        # Sum over all x of (-1)^(x·(s⊕z))
        total = sum((-1)**(sum(1 for i in range(n) if format(x, f'0{n}b')[i] == '1' and s_xor_z[i] == '1') % 2) 
                   for x in range(N))
        
        amplitude = total / N
        if abs(amplitude) > 0.01:
            print(f"      α(|{z_bin}⟩) = {amplitude:.4f}" + 
                  (f" ← This is |s⟩!" if z_bin == secret else ""))
    
    # ═══════════════ Final Result ═══════════════
    print(f"\n" + "═"*65)
    print(f"✓ The sum Σ_x (-1)^(x·(s⊕z)) = {N} only when s⊕z = 0 (i.e., z = s)")
    print(f"✓ All other amplitudes cancel to 0 (destructive interference)")
    print(f"✓ Measuring gives |{secret}⟩ with 100% probability!")
    print("═"*65)


# Run the trace for a secret string
trace_bv_evolution("101")

print("\n\n")
trace_bv_evolution("1101")

### 5.5.3 The Phase Kickback Mechanism (From L1.6)

The lecture explains the **phase kickback trick** in detail. Let's visualize what happens at each CNOT:

**Setup**: Control qubit in |+⟩ = (|0⟩ + |1⟩)/√2, Target qubit in |−⟩ = (|0⟩ - |1⟩)/√2

**Before CNOT**: $|+\rangle|{-}\rangle = \frac{1}{2}(|0\rangle + |1\rangle)(|0\rangle - |1\rangle)$

**After CNOT**:
- When control = |0⟩: target unchanged → |0⟩(|0⟩ - |1⟩)
- When control = |1⟩: target flipped → |1⟩(|1⟩ - |0⟩) = **-|1⟩**(|0⟩ - |1⟩)

**Result**: $\frac{1}{2}(|0\rangle - |1\rangle)(|0\rangle - |1\rangle) = |{-}\rangle|{-}\rangle$

From L1.6: "The control line got flipped [in phase]. This is in some sense opposite of what you were aware of."

The target |−⟩ stays unchanged, but the control qubit picks up a phase!

In [ ]:
# Demonstrate phase kickback with Qiskit
def demonstrate_phase_kickback():
    """
    Show phase kickback: CNOT with |−⟩ target flips the control's phase.
    
    From L1.6: "I was sending in minus here and 0 here. The target line 
    just went through without any change... but my control line got flipped."
    """
    print("="*60)
    print("PHASE KICKBACK DEMONSTRATION")
    print("="*60)
    
    # Case 1: Control = |+⟩, Target = |−⟩
    qc = QuantumCircuit(2)
    qc.h(0)  # Control in |+⟩
    qc.x(1)  # Target will be |−⟩
    qc.h(1)  # Target now |−⟩
    
    state_before = Statevector(qc)
    print(f"\nBEFORE CNOT:")
    print(f"  Control (q0): |+⟩ = (|0⟩ + |1⟩)/√2")
    print(f"  Target (q1):  |−⟩ = (|0⟩ - |1⟩)/√2")
    
    # Apply CNOT
    qc.cx(0, 1)
    state_after = Statevector(qc)
    
    print(f"\nAFTER CNOT:")
    print(f"  State vector: {state_after}")
    
    # Apply H to control to see phase
    qc.h(0)
    final_state = Statevector(qc)
    
    print(f"\nAFTER H on control (to decode phase):")
    print(f"  State vector: {final_state}")
    print(f"  Control measured as: |1⟩ (phase was flipped!)")
    print(f"  Target unchanged: still |−⟩")
    
    # Case 2: Without CNOT (baseline)
    qc_baseline = QuantumCircuit(2)
    qc_baseline.h(0)
    qc_baseline.x(1)
    qc_baseline.h(1)
    # NO CNOT
    qc_baseline.h(0)
    baseline_state = Statevector(qc_baseline)
    
    print(f"\n(Baseline without CNOT: Control → |0⟩)")
    
    return state_after


demonstrate_phase_kickback()

# Show how multiple CNOTs accumulate phase
print("\n\n")
print("="*60)
print("MULTIPLE CNOTS → ACCUMULATED PHASE KICKBACK")
print("="*60)
print("\nFor secret s = 101:")
print("  CNOT at position 0 (s₀=1): flips phase if x₀=1")
print("  No CNOT at position 1 (s₁=0): no flip")
print("  CNOT at position 2 (s₂=1): flips phase if x₂=1")
print("\nTotal phase flip = (-1)^(count of positions where sᵢ=1 AND xᵢ=1)")
print("                 = (-1)^(s·x mod 2)")

### 5.5.4 Interactive Secret String Calculator

Use this tool to explore why measuring gives |s⟩ with 100% probability.

In [ ]:
def explore_bv_amplitude(secret: str, output_z: str):
    """
    Calculate the amplitude of measuring |z⟩ when the secret is s.
    
    Shows why α(|z⟩) = 1 when z = s, and α(|z⟩) = 0 otherwise.
    """
    if len(secret) != len(output_z):
        print("❌ Secret and output must have same length!")
        return
    
    n = len(secret)
    N = 2**n
    
    print(f"Computing amplitude of |{output_z}⟩ when secret s = {secret}")
    print("=" * 60)
    
    # Compute s XOR z
    s_xor_z = ''.join('1' if secret[i] != output_z[i] else '0' for i in range(n))
    print(f"s ⊕ z = {secret} ⊕ {output_z} = {s_xor_z}")
    
    print(f"\nα(|{output_z}⟩) = (1/{N}) Σ_x (-1)^(x·(s⊕z))")
    print(f"            = (1/{N}) Σ_x (-1)^(x·{s_xor_z})")
    print()
    
    # Sum over all x
    total = 0
    print("Sum breakdown:")
    for x in range(min(N, 16)):
        x_bin = format(x, f'0{n}b')
        # Compute x·(s⊕z)
        dot_product = sum(1 for i in range(n) if x_bin[i] == '1' and s_xor_z[i] == '1') % 2
        contribution = (-1) ** dot_product
        total += contribution
        sign = "+" if contribution > 0 else "-"
        print(f"  x={x_bin}: x·{s_xor_z} = {dot_product} → {sign}1")
    
    if N > 16:
        # Complete the sum for larger N
        total = sum((-1)**(sum(1 for i in range(n) if format(x, f'0{n}b')[i] == '1' and s_xor_z[i] == '1') % 2) 
                   for x in range(N))
        print(f"  ... ({N-16} more terms)")
    
    print("-" * 60)
    amplitude = total / N
    print(f"Sum = {total}")
    print(f"Amplitude = {total}/{N} = {amplitude}")
    
    if output_z == secret:
        print(f"\n✓ z = s, so s⊕z = {'0'*n} → all terms are +1 → Sum = {N} → Amplitude = 1")
    else:
        print(f"\n✓ z ≠ s, so s⊕z ≠ {'0'*n} → half +1, half -1 → Sum = 0 → Amplitude = 0")


# Example 1: When z = s (should get amplitude 1)
print("="*60)
print("CASE 1: Measuring the secret string itself")
print("="*60)
explore_bv_amplitude("101", "101")

print("\n\n")

# Example 2: When z ≠ s (should get amplitude 0)
print("="*60)
print("CASE 2: Measuring a different string")
print("="*60)
explore_bv_amplitude("101", "100")

### 5.5.5 📊 Summary: Key Takeaways

| Stage | Mathematical State | Physical Meaning |
|-------|-------------------|------------------|
| **① Init** | $|0\rangle^{\otimes n} \otimes |1\rangle$ | Start with ancilla in $|1\rangle$ |
| **② After H** | $\frac{1}{\sqrt{2^n}} \sum_x |x\rangle \otimes |{-}\rangle$ | Uniform superposition + phase-sensitive ancilla |
| **③ After Oracle** | $\frac{1}{\sqrt{2^n}} \sum_x (-1)^{s \cdot x} |x\rangle \otimes |{-}\rangle$ | **Phase kickback**: $s \cdot x$ encoded in phases |
| **④ After Final H** | $|s\rangle \otimes |{-}\rangle$ | **Perfect interference**: only $|s\rangle$ survives |

**The 1-Matching Rule**: Sign of $|x\rangle$ after oracle = $(-1)^{\text{count of positions where both } s_i=1 \text{ and } x_i=1}$

**The Magic**: 
- When $z = s$: $s \oplus z = 0$, so $x \cdot (s \oplus z) = 0$ for all $x$ → all $2^n$ terms are $+1$ → **constructive** interference
- When $z \neq s$: $s \oplus z \neq 0$, so half the $x$ give $+1$, half give $-1$ → **destructive** interference (cancellation to 0)

**Quantum Advantage**: 
- Classical: Need $n$ queries (one per bit of $s$)
- Quantum: Need exactly **1** query
- Speedup: $n \times$ (linear, not exponential!)

**Connection to Deutsch-Jozsa**:
BV is essentially the "inverse" of DJ:
- DJ: Given oracle, determine if function is constant or balanced
- BV: Given that function is $s \cdot x \mod 2$, find the secret $s$

Both use the **same circuit structure** (H-Oracle-H) and rely on the **same phase kickback mechanism**!

## Section 6: Classical Comparison

In [ ]:
def classical_bv(oracle_func, n: int) -> Tuple[str, int]:
    """
    Classical algorithm to find secret using n queries.
    
    Strategy: Query with unit vectors e_i = 00...010...0 (1 at position i)
    Then f(e_i) = s·e_i = s_i (reveals one bit of s)
    
    Args:
        oracle_func: Function implementing f(x) = s·x mod 2
        n: Number of bits
    
    Returns:
        (discovered_secret, number_of_queries)
    """
    queries = 0
    secret_bits = []
    
    for i in range(n):
        # Create unit vector e_i (1 at position i from the right)
        e_i = '0' * (n - 1 - i) + '1' + '0' * i
        
        # Query oracle
        result = oracle_func(e_i)
        queries += 1
        
        # This reveals s_i
        secret_bits.append(str(result))
    
    discovered_secret = ''.join(secret_bits)
    return discovered_secret, queries


def create_classical_oracle(secret: str):
    """
    Create a classical oracle function for f(x) = s·x mod 2.
    """
    def oracle(x: str) -> int:
        result = 0
        for s_bit, x_bit in zip(secret, x):
            result ^= (int(s_bit) & int(x_bit))
        return result
    return oracle


# Compare classical and quantum approaches
print("Classical vs Quantum: Finding the Secret")
print("="*60)

test_secrets = ["10", "101", "1101", "11010", "101010"]

for secret in test_secrets:
    n = len(secret)
    
    # Classical approach
    oracle_func = create_classical_oracle(secret)
    classical_result, classical_queries = classical_bv(oracle_func, n)
    
    # Quantum approach
    quantum_result, _ = find_secret(secret)
    quantum_queries = 1  # Always 1 for BV
    
    print(f"\nSecret: {secret}")
    print(f"  Classical: {classical_result} in {classical_queries} queries")
    print(f"  Quantum:   {quantum_result} in {quantum_queries} query")
    print(f"  Speedup:   {classical_queries}x")

In [ ]:
# Visualize the speedup
n_values = list(range(1, 12))
classical_queries = n_values  # n queries for n bits
quantum_queries = [1] * len(n_values)  # Always 1 query

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
axes[0].plot(n_values, classical_queries, 'ro-', label='Classical (n queries)', linewidth=2, markersize=8)
axes[0].plot(n_values, quantum_queries, 'b^-', label='Quantum (1 query)', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of bits (n)')
axes[0].set_ylabel('Number of queries')
axes[0].set_title('Query Complexity: BV Algorithm')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(n_values)

# Speedup
speedup = [c / q for c, q in zip(classical_queries, quantum_queries)]
axes[1].bar(n_values, speedup, color='purple', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Number of bits (n)')
axes[1].set_ylabel('Speedup (n/1 = n)')
axes[1].set_title('Quantum Speedup Factor')
axes[1].set_xticks(n_values)

plt.tight_layout()
plt.show()

print("\n📊 Key Insight: BV provides LINEAR speedup (n → 1)")
print("   This is less dramatic than exponential, but still significant!")

## Section 7: Common Traps Demonstration

In [ ]:
# TRAP 1: Bit ordering confusion
print("="*60)
print("TRAP 1: Bit Ordering Confusion")
print("="*60)

def create_bv_oracle_BUGGY(secret: str) -> QuantumCircuit:
    """BUGGY: Doesn't reverse the secret string."""
    n = len(secret)
    oracle = QuantumCircuit(n + 1, name=f"Buggy_Oracle")
    
    # BUG: Not reversing the string!
    for i, bit in enumerate(secret):  # Should be enumerate(reversed(secret))
        if bit == '1':
            oracle.cx(i, n)
    
    return oracle

secret = "100"  # MSB=1, LSB=0

# Correct oracle
correct_oracle = create_bv_oracle(secret)
correct_circuit = QuantumCircuit(4, 3)
correct_circuit.x(3)
correct_circuit.h(range(4))
correct_circuit.compose(correct_oracle, inplace=True)
correct_circuit.h(range(3))
correct_circuit.measure(range(3), range(3))

# Buggy oracle
buggy_oracle = create_bv_oracle_BUGGY(secret)
buggy_circuit = QuantumCircuit(4, 3)
buggy_circuit.x(3)
buggy_circuit.h(range(4))
buggy_circuit.compose(buggy_oracle, inplace=True)
buggy_circuit.h(range(3))
buggy_circuit.measure(range(3), range(3))

# Use SamplerV2 with FakeManilaV2 backend
sampler = SamplerV2()
backend = FakeManilaV2()

# Transpile circuits
correct_circuit_transpiled = transpile(correct_circuit, backend=backend, optimization_level=1)
buggy_circuit_transpiled = transpile(buggy_circuit, backend=backend, optimization_level=1)

# Run circuits
correct_job = sampler.run([correct_circuit_transpiled], shots=1)
buggy_job = sampler.run([buggy_circuit_transpiled], shots=1)

correct_result_data = correct_job.result()
buggy_result_data = buggy_job.result()

correct_result = list(correct_result_data[0].data.c.get_counts().keys())[0]
buggy_result = list(buggy_result_data[0].data.c.get_counts().keys())[0]

print(f"\nSecret: {secret}")
print(f"Correct oracle finds: {correct_result} {'✓' if correct_result == secret else '✗'}")
print(f"Buggy oracle finds:   {buggy_result} {'✓' if buggy_result == secret else '✗'}")
print(f"\n💡 The buggy oracle returned the REVERSED secret!")
print(f"   Because it mapped s[0] to q_0 instead of q_{len(secret)-1}.")

In [ ]:
# TRAP 2: Confusing BV with DJ
print("\n" + "="*60)
print("TRAP 2: Confusing BV with DJ")
print("="*60)

# Same circuit, different interpretation!
secret = "110"
circuit = bernstein_vazirani(secret)

# Use SamplerV2 with FakeManilaV2 backend
sampler = SamplerV2()
backend = FakeManilaV2()
# Transpile circuits
transpiled = transpile(correct_circuit, backend=backend, optimization_level=1)

result = sampler.run([transpiled], shots=1).result()
measurement = list(result[0].data.c.get_counts().keys())[0]

print(f"\nCircuit measurement: {measurement}")
print(f"\n🔵 Deutsch-Jozsa interpretation:")
if measurement == "0" * len(secret):
    print(f"   Output is all zeros → Function is CONSTANT")
else:
    print(f"   Output is not all zeros → Function is BALANCED")

print(f"\n🟢 Bernstein-Vazirani interpretation:")
print(f"   Output {measurement} IS the secret string s!")

print(f"\n💡 Same circuit, different problems:")
print(f"   - DJ asks: 'Is f constant or balanced?'")
print(f"   - BV asks: 'What is the hidden string s?'")

## Section 8: Noise Analysis

In [ ]:
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer import AerSimulator

def test_bv_with_noise(secret: str, error_rate: float, shots: int = 1000) -> float:
    """
    Test BV algorithm under depolarizing noise.
    Note: Noise simulation still uses AerSimulator as SamplerV2 doesn't support custom noise models.
    
    Returns:
        Probability of correctly measuring the secret
    """
    circuit = bernstein_vazirani(secret)
    backend = FakeManilaV2()
    
    # Create noise model
    noise_model = NoiseModel()
    error_1q = depolarizing_error(error_rate, 1)
    error_2q = depolarizing_error(error_rate * 5, 2)  # 2-qubit gates have higher error
    noise_model.add_all_qubit_quantum_error(error_1q, ['h', 'x'])
    noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])
    
    # Transpile circuit
    transpiled_circuit = transpile(circuit, backend=backend, optimization_level=1)
    
    # Run simulation with noise (AerSimulator needed for noise model)
    noisy_sim = AerSimulator(noise_model=noise_model)
    result = noisy_sim.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    
    # Calculate accuracy
    correct_count = counts.get(secret, 0)
    accuracy = correct_count / shots
    
    return accuracy, counts


# Test across error rates
secret = "1011"
error_rates = [0, 0.001, 0.005, 0.01, 0.02, 0.05, 0.1]
accuracies = []

print(f"BV Accuracy vs Error Rate (secret = {secret})")
print("="*50)

for error in error_rates:
    acc, _ = test_bv_with_noise(secret, error)
    accuracies.append(acc)
    print(f"Error rate: {error*100:5.1f}% → Accuracy: {acc*100:5.1f}%")

# Plot
plt.figure(figsize=(10, 6))
plt.plot([e*100 for e in error_rates], [a*100 for a in accuracies], 'bo-', linewidth=2, markersize=10)
plt.axhline(y=50, color='r', linestyle='--', label='Random guessing threshold')
plt.xlabel('Gate Error Rate (%)')
plt.ylabel('Accuracy (%)')
plt.title(f'BV Algorithm Accuracy Under Noise (secret = {secret})')
plt.legend()

plt.grid(True, alpha=0.3)
print("   Even with 5% error, the correct answer is still most likely.")

plt.ylim(0, 105)
print("\n💡 Observation: BV degrades gracefully with noise.")

plt.show()

In [ ]:
# Detailed histogram for noisy results
secret = "101"
error_rate = 0.02

_, counts = test_bv_with_noise(secret, error_rate, shots=2000)

# Sort by count
sorted_counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))

plt.figure(figsize=(12, 5))
labels = list(sorted_counts.keys())
values = [v/2000 for v in sorted_counts.values()]
colors = ['green' if l == secret else 'lightblue' for l in labels]

plt.bar(range(len(labels)), values, color=colors, edgecolor='black')
plt.xticks(range(len(labels)), labels, rotation=45)
plt.xlabel('Measurement Outcome')
plt.ylabel('Probability')
plt.title(f'BV with 2% Noise (secret = {secret})')
plt.tight_layout()
plt.show()

print(f"\n📊 Even with noise, {secret} (green) has highest probability.")
print(f"   In practice, we'd take majority vote over multiple shots.")

## Section 9: Exercises

### Exercise 1: Hand Trace (Beginner)
For secret s = 10, trace through the BV algorithm showing the state after each step.

In [ ]:
# TODO: Exercise 1
# Use visualize_bv_evolution("10") and verify the math by hand

# Your work here:

### Exercise 2: Oracle Black Box (Intermediate)
Implement a "blind" BV where you DON'T know the secret but receive an oracle. Use BV to discover it.

In [ ]:
# TODO: Exercise 2
# 1. Generate a random 5-bit secret
# 2. Create the oracle (but pretend you don't know the secret)
# 3. Run BV to discover the secret
# 4. Verify by comparing with the actual secret

import random

# Generate random secret (this would normally be unknown)
n = 5
true_secret = ''.join([str(random.randint(0, 1)) for _ in range(n)])
oracle = create_bv_oracle(true_secret)

# TODO: Use only 'oracle' and 'n' to find the secret
# discovered_secret = ...

### Exercise 3: Error Threshold (Advanced)
Find the maximum error rate at which BV still correctly identifies the secret >90% of the time for n=4.

In [ ]:
# TODO: Exercise 3
# Binary search for the threshold error rate

# Your code here:

## Section 10: Quick Knowledge Check

**Q1**: Why does BV need only one query while classical needs n?

<details>
<summary>Click for answer</summary>
Classical must query unit vectors one at a time, each revealing one bit. Quantum creates superposition of ALL inputs, and the Hadamard transform decodes the phase pattern directly into |s⟩, revealing all bits simultaneously.
</details>

**Q2**: If s = 0000, what does BV return?

<details>
<summary>Click for answer</summary>
BV returns 0000. The oracle does nothing (no CNOTs), so H→Identity→H = Identity on input qubits. The state |0000⟩ after the first H transforms back to |0000⟩ after the second H.
</details>

**Q3**: How is BV useful for real applications?

<details>
<summary>Click for answer</summary>
BV itself isn't practically useful (we rarely have such specific oracle access). Its value is educational: it demonstrates quantum speedup for information extraction and introduces techniques used in more powerful algorithms like Simon's and Shor's.
</details>

## Section 11: Summary & Next Steps

### Key Takeaways

1. **Same circuit, different problem**: BV uses DJ's circuit to extract a hidden string
2. **Hadamard as decoder**: Converts phase pattern (-1)^(s·x) into amplitude at |s⟩
3. **Linear speedup**: n queries → 1 query (linear, not exponential)
4. **Bit ordering matters**: Qiskit uses LSB ordering; reverse the secret for correct mapping
5. **Noise tolerance**: BV degrades gracefully; majority vote helps in practice

### Connection to Other Algorithms

| Algorithm | Problem | Speedup |
|-----------|---------|--------|
| Deutsch-Jozsa | Constant vs Balanced | Exponential (vs deterministic) |
| **Bernstein-Vazirani** | **Find hidden string** | **Linear (n → 1)** |
| Simon's | Find hidden period | Exponential |
| Shor's | Factor integers | Exponential |

### What's Next?

- **Module 7.3**: Simon's Algorithm (exponential speedup for period finding)
- **Module 7.4**: Grover's Algorithm (quadratic speedup for search)
- **Module 7.5**: Introduction to Shor's Algorithm